In [1]:
# Audio processing
import librosa
# import noisereduce
# Data cleaning
import os
import pandas as pd
import numpy as np
from numpy import mean, var
# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns
# Statistics
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import train_test_split
# Advanced options
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

In [2]:
FRAMESIZE = 1024
HOPLENGTH = 512
MELS = 13
PCA_N = 2

In [3]:
# Cluster data for future work
opd_data = []
others_data = []
directory= "A:\Professional\Engineering CU\DSP_Data_New"
for dir in os.listdir(directory):
    if dir == "open the door":
        for filename in os.listdir(os.path.join(directory, dir)):
            path_opd = os.path.join(directory, dir, filename)
            opd_data.append(path_opd)
    else:
        for filename in os.listdir(os.path.join(directory, dir)):
            path_others = os.path.join(directory, dir, filename)
            others_data.append(path_others)

In [4]:
# Group all the data
data = []
data.extend(opd_data)
data.extend(others_data)

In [13]:
# Fingerprint Dataframe
password_open = pd.DataFrame()
password_others =pd.DataFrame()

password_open['data'] = opd_data
password_others['data'] = others_data
password_open['pass'] = 1
password_others['pass'] = 0

password = password_open.append(password_others, ignore_index=True)
password

,data,pass
0,A:\Professional\Engineering CU\DSP_Data_New\op...,1
1,A:\Professional\Engineering CU\DSP_Data_New\op...,1
2,A:\Professional\Engineering CU\DSP_Data_New\op...,1
3,A:\Professional\Engineering CU\DSP_Data_New\op...,1
4,A:\Professional\Engineering CU\DSP_Data_New\op...,1
...,...,...
187,A:\Professional\Engineering CU\DSP_Data_New\un...,0
188,A:\Professional\Engineering CU\DSP_Data_New\un...,0
189,A:\Professional\Engineering CU\DSP_Data_New\un...,0
190,A:\Professional\Engineering CU\DSP_Data_New\un...,0


In [14]:
# Functions we will use
def transform_audio(audio, FRAMESIZE, HOPLENGTH, MELS):

    # audio_noised, sr = librosa.load(audio, duration=2)
    # audio_array = noisereduce.reduce_noise(y=audio_noised, sr=sr)
    audio_array, sr = librosa.load(audio, duration=2)

    log_mel_audio_list_mean = []
    log_mel_audio_list_var = []
    mfccs_audio_list_mean = []
    mfccs_audio_list_var = []
    cqt_audio_list_mean = []
    cqt_audio_list_var = []
    chromagram_audio_list_mean = []
    chromagram_audio_list_var = []
    tone_audio_list_mean = []
    tone_audio_list_var = []

    log_mel_audio = librosa.power_to_db(librosa.feature.melspectrogram(audio_array, sr=sr, n_fft=FRAMESIZE, hop_length=HOPLENGTH, n_mels=MELS))
    mfccs_audio = librosa.feature.mfcc(y=audio_array, n_mfcc=MELS, sr=sr, n_fft=FRAMESIZE, hop_length=HOPLENGTH)

    cqt_audio = np.abs(librosa.cqt(y=audio_array, sr=sr, hop_length=HOPLENGTH))
    chromagram_audio = librosa.feature.chroma_stft(audio_array, sr=sr, n_fft=FRAMESIZE, hop_length=HOPLENGTH)
    tone_audio = librosa.feature.tonnetz(y=audio_array, sr=sr)

    for i in range(len(log_mel_audio)):
         log_mel_audio_list_mean.append(log_mel_audio[i].mean())
         log_mel_audio_list_var.append(log_mel_audio[i].var())

    for i in range(len(mfccs_audio)):
         mfccs_audio_list_mean.append(mfccs_audio[i].mean())
         mfccs_audio_list_var.append(mfccs_audio[i].var())

    for i in range(len(cqt_audio)):
         cqt_audio_list_mean.append(cqt_audio[i].mean())
         cqt_audio_list_var.append(cqt_audio[i].var())

    for i in range(len(chromagram_audio)):
         chromagram_audio_list_mean.append(chromagram_audio[i].mean())
         chromagram_audio_list_var.append(chromagram_audio[i].var())

    for i in range(len(tone_audio)):
         tone_audio_list_mean.append(tone_audio[i].mean())
         tone_audio_list_var.append(tone_audio[i].var())

    sb_audio = librosa.feature.spectral_bandwidth(y=audio_array, sr=sr, n_fft=FRAMESIZE, hop_length=HOPLENGTH)

    ae_audio = fancy_amplitude_envelope(audio_array, FRAMESIZE, HOPLENGTH)
    rms_audio = librosa.feature.rms(audio_array, frame_length=FRAMESIZE, hop_length=HOPLENGTH)

    return np.hstack((mean(ae_audio), var(ae_audio), mean(rms_audio), var(rms_audio), mean(sb_audio), var(sb_audio), chromagram_audio_list_mean, chromagram_audio_list_var, tone_audio_list_mean, tone_audio_list_var, cqt_audio_list_mean, cqt_audio_list_var, mfccs_audio_list_mean, mfccs_audio_list_var, log_mel_audio_list_mean, log_mel_audio_list_var))


def fancy_amplitude_envelope(signal, framesize, hoplength):
    return np.array([max(signal[i:i+framesize]) for i in range(0, len(signal), hoplength)])

In [15]:
samples_no = len(password['data'])
features_length = len(transform_audio(password['data'].iloc[0], FRAMESIZE, HOPLENGTH, MELS))

In [20]:
features_df = pd.DataFrame()
features_df['data'] = password['data'].copy()
features_df['pass'] = password['pass'].copy()

for i in tqdm(range(features_length)):
    features_df[f'feature {i}'] = np.zeros(samples_no)

temp_array = []
for index, row in tqdm(features_df.iloc[:].iterrows()):
    if index < len(features_df)-1:
        array_1 = transform_audio(features_df.loc[index, ['data']][0], FRAMESIZE, HOPLENGTH, MELS)
        array_2 = transform_audio(features_df.loc[index+1, ['data']][0], FRAMESIZE, HOPLENGTH, MELS)
        combined_array = np.vstack((array_1, array_2))[0]
        temp_array.append(list(combined_array))

last_element = transform_audio(features_df.loc[len(features_df)-1, ['data']][0], FRAMESIZE, HOPLENGTH, MELS)
temp_array.append(list(last_element))


features_df.iloc[:,2:] = pd.DataFrame(temp_array).copy()
features_df

100%|██████████| 262/262 [00:00<00:00, 4635.33it/s]
192it [01:09,  2.76it/s]


,data,pass,feature 0,feature 1,feature 2,feature 3,feature 4,feature 5,feature 6,feature 7,...,feature 252,feature 253,feature 254,feature 255,feature 256,feature 257,feature 258,feature 259,feature 260,feature 261
0,A:\Professional\Engineering CU\DSP_Data_New\op...,1,0.027843,0.002087,0.013634,0.000580,2409.485534,444727.497430,0.637220,0.566248,...,216.462494,176.181519,128.056839,149.693787,209.715164,148.801743,85.350876,59.285122,70.233040,65.921082
1,A:\Professional\Engineering CU\DSP_Data_New\op...,1,0.033648,0.004332,0.015390,0.000874,2302.179488,505915.237556,0.475199,0.436228,...,219.727203,170.745605,97.705536,127.613663,160.602402,135.815109,92.522125,74.727669,85.024757,104.147827
2,A:\Professional\Engineering CU\DSP_Data_New\op...,1,0.023385,0.001250,0.013316,0.000455,2280.717985,609810.794167,0.515015,0.428426,...,211.628494,204.353867,129.512207,116.327835,165.632950,190.573883,107.446518,43.292130,53.123344,51.931469
3,A:\Professional\Engineering CU\DSP_Data_New\op...,1,0.024663,0.000743,0.007930,0.000077,3207.467542,21074.717748,0.666652,0.625858,...,33.583759,28.132874,14.423258,5.036216,11.231256,8.625013,3.750520,0.598897,0.544173,0.362404
4,A:\Professional\Engineering CU\DSP_Data_New\op...,1,0.051932,0.020891,0.020185,0.001819,2241.470457,627994.777186,0.452060,0.515571,...,271.744171,223.154694,169.702164,114.854019,138.567413,167.287033,108.930367,58.179504,53.992485,42.848633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,A:\Professional\Engineering CU\DSP_Data_New\un...,0,0.015773,0.000217,0.005335,0.000030,3280.455819,22913.322885,0.569803,0.590974,...,39.137447,33.027950,7.295224,2.550004,4.270374,2.958375,1.051028,0.757277,0.480075,0.356831
188,A:\Professional\Engineering CU\DSP_Data_New\un...,0,0.047651,0.005682,0.018710,0.000937,2221.252490,696581.265398,0.534416,0.452144,...,260.199554,288.955322,152.535645,164.240067,200.829163,171.758636,178.650635,59.455601,42.259415,65.068863
189,A:\Professional\Engineering CU\DSP_Data_New\un...,0,0.051047,0.005127,0.019362,0.000863,2960.785129,289192.140017,0.536324,0.534323,...,154.288620,100.381271,21.474648,64.361992,51.115452,39.914986,15.535128,1.903714,2.089182,1.833574
190,A:\Professional\Engineering CU\DSP_Data_New\un...,0,0.039748,0.004213,0.015745,0.000659,2328.297810,688890.333144,0.527444,0.472268,...,273.135925,274.134705,121.783401,157.892990,211.429337,204.492920,150.704941,35.732243,24.178911,42.259094


In [22]:
features_df.iloc[:,2:].to_csv("..\\..\\data\\password_data.csv", index=False)
features_df['pass'].to_csv('..\\..\\data\\password_target.csv', index=False)